In [ ]:
repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'

data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'

import numpy as np
import sys

# graph package
import igraph as ig

# our code
sys.path.append(repo_directory + 'code/')
from setup_data_dir import setup_data_dir, make_subnetwork_directory
from pipeline.download_data import download_bulk_resource, download_master_edgelist, download_scdb
from helpful_functions import case_info

sys.path.append(repo_directory + 'explore/vertex_metrics_experiment/code/')
from rankscore_experiment_sort import *
from rankscore_experiment_LR import *
from make_tr_edge_df import *


# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

## parameters from make snapshots

In [ ]:
metrics = ['age', 'indegree', 'outdegree', 'degree', 'd_pagerank', 'authorities', 'hubs']

# add recent citations
metrics += ['recentcite_' + str(t) for t in 5 * np.arange(1, 6+1)]

active_years = range(1900, 2015 + 1)

## test parameters

In [ ]:
test_params = {'active_years':  range(1900, 2015 + 1),
               'seed': 4332,
               'num_test_cases': 1000}

# log reg parameters

In [ ]:
# how many abset edges to add
num_absent_edges = len(G.es)
seed_edge_df = 32432

# include similarity in logistic regrssion
include_similarity = True

# how to normalize yearly metrics
metric_normalization = 'mean'

# rank by sorting

In [ ]:
scores_sort = get_rankscores_sort(G, test_params, metrics,
                                  subnet_dir,
                                  print_progress=True)

In [ ]:
scores_sort.mean()

# logistic regression

## make training data for logistic regression

In [ ]:
%time make_tr_edge_df(G, subnet_dir, \
                      active_years, num_absent_edges,\
                      metrics, metric_normalization=metric_normalization,\
                      seed=seed_edge_df)

In [ ]:
tr_edge_data = pd.read_csv(subnet_dir + 'edge_data.csv',
                           index_col=0)

# rank by logistic regression

In [ ]:
%time scores_LR = get_rankscores_LR(G, test_params, metrics, \
                                    include_similarity, subnet_dir,\
                                    metric_normalization=metric_normalization, print_progress=True) 
                                    

# surgery